In [1]:
import requests
import json
import string
from scrapy.http import HtmlResponse





In [2]:
url = f"https://www.sebi.gov.in/sebiweb/ajax/home/getnewslistallinfo.jsp"

# payload = "companyname=Idea Engine"
payload = {"nextValue": "2"}
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
    # "authority": "www.mca.gov.in",
    # "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
}

res = requests.post(url, headers=headers, data=payload)

In [4]:
for i in range(1,2225):
    
    url = f"https://www.sebi.gov.in/sebiweb/ajax/home/getnewslistallinfo.jsp"

    # payload = "companyname=Idea Engine"
    payload = {"nextValue": f"{i}"}
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        # "authority": "www.mca.gov.in",
        # "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
    }
    res = requests.post(url, headers=headers, data=payload)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [3]:
res.content

b'<input type=\'hidden\' name=\'totalpage\' value=2224 /><input type=\'hidden\' name=\'nextDel\' value=\'25\'/><input type=\'hidden\' name=\'nextValue\' value=\'0\'/><div class=\'pagination\'><div class=\'pagination_inner\'><p>&nbsp;51 to 75 of 55599 records</p></div><div class=\'pagination_outer\'><ul><li><a href="javascript: searchFormNewsListAll(\'n\', \'0\');" title=\'First\'><i class=\'fa fa-angle-double-left\'></i></a></li><li><a href="javascript: searchFormNewsListAll(\'p\',\'-1\')" title=\'Prev\'><i class=\'fa fa-angle-left\'></i></a></li><li><a href="javascript: searchFormNewsListAll(\'n\', \'0\');">1</a></li><li><a href="javascript: searchFormNewsListAll(\'n\', \'1\');">2</a></li><li><a class=\'active\'>3</a></li><li><a href="javascript: searchFormNewsListAll(\'n\', \'3\');">4</a></li><li><a href="javascript: searchFormNewsListAll(\'n\', \'4\');">5</a></li><li><a href="javascript: searchFormNewsListAll(\'n\', \'5\');">6</a></li><li><a href="javascript: searchFormNewsListAll(\

In [43]:
import string
from string import punctuation
l = list(string.ascii_lowercase) + [str(num) for num in list(range(10))]  + list(punctuation)
ml = [] 
for i in l:
    for k in l:
        for j in l:
            ml.append(i+k+j)

In [44]:
ml

314432

In [39]:
len(ml)

46656

In [34]:
ml

['aaa',
 'aab',
 'aac',
 'aad',
 'aae',
 'aaf',
 'aag',
 'aah',
 'aai',
 'aaj',
 'aak',
 'aal',
 'aam',
 'aan',
 'aao',
 'aap',
 'aaq',
 'aar',
 'aas',
 'aat',
 'aau',
 'aav',
 'aaw',
 'aax',
 'aay',
 'aaz',
 'aba',
 'abb',
 'abc',
 'abd',
 'abe',
 'abf',
 'abg',
 'abh',
 'abi',
 'abj',
 'abk',
 'abl',
 'abm',
 'abn',
 'abo',
 'abp',
 'abq',
 'abr',
 'abs',
 'abt',
 'abu',
 'abv',
 'abw',
 'abx',
 'aby',
 'abz',
 'aca',
 'acb',
 'acc',
 'acd',
 'ace',
 'acf',
 'acg',
 'ach',
 'aci',
 'acj',
 'ack',
 'acl',
 'acm',
 'acn',
 'aco',
 'acp',
 'acq',
 'acr',
 'acs',
 'act',
 'acu',
 'acv',
 'acw',
 'acx',
 'acy',
 'acz',
 'ada',
 'adb',
 'adc',
 'add',
 'ade',
 'adf',
 'adg',
 'adh',
 'adi',
 'adj',
 'adk',
 'adl',
 'adm',
 'adn',
 'ado',
 'adp',
 'adq',
 'adr',
 'ads',
 'adt',
 'adu',
 'adv',
 'adw',
 'adx',
 'ady',
 'adz',
 'aea',
 'aeb',
 'aec',
 'aed',
 'aee',
 'aef',
 'aeg',
 'aeh',
 'aei',
 'aej',
 'aek',
 'ael',
 'aem',
 'aen',
 'aeo',
 'aep',
 'aeq',
 'aer',
 'aes',
 'aet',
 'aeu',


In [33]:
len(ml)

17576

In [16]:
import json
data = json.load(open("MCA-ALL/mca-all-digit.json"))

In [17]:
len(data)

830413

In [18]:
data[0]

{'companyID': 'U45200TG1997PTC027764',
 'companyName': 'AAA DEVELOPERS PRIVATE LIMITED'}

In [30]:
cids = []
cnames = []
for i in data:
    cids.append(i["companyID"])
    cnames.append(i["companyName"])

In [31]:
print(len(list(set(cids))))
print(len(list(set(cnames))))


799567
799153


In [21]:
import pandas as pd
raw_data = pd.DataFrame(data)

In [22]:
raw_data

,companyID,companyName
0,U45200TG1997PTC027764,AAA DEVELOPERS PRIVATE LIMITED
1,U65992HR1996PTC033148,AAA CHITS PVT. LTD.
2,U18101KL2005PTC018696,AAA APPARELS PRIVATE LIMITED
3,U00452MP1995PTC009503,AAA HOUSING FINANCE PVT LTD
4,U52599MP2002PTC015075,AAA NETWORK DISTRIBUTIONS PRIVATE LIMITED
...,...,...
830408,U70200HR2021PTC097871,999 TECHNOLOGY VENTURES PRIVATE LIMITED
830409,U51399TG2021PTC155515,999 PLYWOODBASKET.COM PRIVATE LIMITED
830410,U71290MH2022PTC376317,999 LEASING & INFRASTRUCTURE PRIVATE LIMITED
830411,AAF-8312,999 CREATIVE SERVICES LLP


In [23]:
r_d = raw_data.drop_duplicates()

In [24]:
r_d

,companyID,companyName
0,U45200TG1997PTC027764,AAA DEVELOPERS PRIVATE LIMITED
1,U65992HR1996PTC033148,AAA CHITS PVT. LTD.
2,U18101KL2005PTC018696,AAA APPARELS PRIVATE LIMITED
3,U00452MP1995PTC009503,AAA HOUSING FINANCE PVT LTD
4,U52599MP2002PTC015075,AAA NETWORK DISTRIBUTIONS PRIVATE LIMITED
...,...,...
830406,U92490MH2020PTC337765,999 RBC ENTERTAINMENT PRIVATE LIMITED
830407,U18109GJ2021PTC119722,999TEE (INDIA) PRIVATE LIMITED
830408,U70200HR2021PTC097871,999 TECHNOLOGY VENTURES PRIVATE LIMITED
830409,U51399TG2021PTC155515,999 PLYWOODBASKET.COM PRIVATE LIMITED


In [27]:
url = f"https://www.mca.gov.in/mcafoportal/companyLLPMasterData.do"
payload = {"companyName": "NAMELESS CREATIVES PRIVATE LIMITED",
"companyID": "U72900DL2017PTC327028",
"displayCaptcha": "true",
"userEnteredCaptcha": "snesors"}
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
    "authority": "www.mca.gov.in",
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
}

res = requests.post(url, headers=headers, data=payload)



In [29]:
res.content

b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>403 Forbidden</title>\n</head><body>\n<h1>Forbidden</h1>\n<p>You don\'t have permission to access /mcafoportal/companyLLPMasterData.do\non this server.</p>\n</body></html>\n'

In [28]:
data = json.loads(res.text)["companyList"]
data

[{'companyID': 'U45200TG1997PTC027764',
  'companyName': 'AAA DEVELOPERS PRIVATE LIMITED'},
 {'companyID': 'U65992HR1996PTC033148', 'companyName': 'AAA CHITS PVT. LTD.'},
 {'companyID': 'U18101KL2005PTC018696',
  'companyName': 'AAA APPARELS PRIVATE LIMITED'},
 {'companyID': 'U00452MP1995PTC009503',
  'companyName': 'AAA HOUSING FINANCE PVT LTD'},
 {'companyID': 'U52599MP2002PTC015075',
  'companyName': 'AAA NETWORK DISTRIBUTIONS PRIVATE LIMITED'},
 {'companyID': 'U65990MH1982PTC026796',
  'companyName': 'AAA INVESTMENT CO PVT LTD'},
 {'companyID': 'U28920MH1988PTC049710',
  'companyName': 'AAA ENGINEERING ENTERPRISES PRIVATE LIMITED'},
 {'companyID': 'U65910MH1991PTC064007',
  'companyName': 'AAA HOSPITALITY SERVICES PVT LTD'},
 {'companyID': 'U65923OR1994PLC003706',
  'companyName': 'AAA INVESTMENT AND FINANCE LTD'},
 {'companyID': 'U65921PB1990PTC010323', 'companyName': 'AAA FINANCE PVT LTD'},
 {'companyID': 'U70109PB2002PTC025120',
  'companyName': 'AAA INFRA VENTURES PRIVATE LIMIT

In [1]:
from deep_translator import GoogleTranslator
def langtranslation(to_translate):
    try:
        translated = GoogleTranslator(source='auto', target='en').translate(to_translate)
    except:
        try:
            translated = GoogleTranslator(source='auto', target='en').translate(to_translate)
        except:
            print(f">>>Translartion Bug : {to_translate}")
            translated = to_translate   
    return translated

# -*- coding: utf-8 -*-
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [46]:
import camelot

In [3]:
tb = camelot.read_pdf("raw-files/file-157415407029476.pdf",pages="1",flavor="stream")
# tb = camelot.read_pdf("raw-files/file-157415407029476.pdf",pages="1",flavor="stream")


In [47]:
main = tb[0].df
new_header = main.iloc[0] #grab the first row for the header
main = main[1:] #take the data less the header row
main.columns = new_header #set the header row as the df header
main

,Arabic Name\n يبرعلاب مسلاا,English Name\n يزيلجنلااب مسلاا,Code\nزمرلا,,Arabic Address\n يبرعلاب ناونعلا,English Address\nيزيلجنلااب ناونعلا
1,يملاسلاا دلابلا,Al Bilad Islamic,BIQ034/2,,ةيرحلا ةحاس برق\n /\n ةداركلا\n /\n دادغب,Baghdad / Karrada / near Al Horriya square
2,يملاسلاا يميلقلاا نواعتلا,Islamic Regional Co-operation,BIQ036/2,,59\n ةيانب\n / \n17\n عراش\n / \n929\n ةلحم\n ...,Baghdad / Al Masbah / section 929 / street 17 ...
3,ناريا يلم,Melli Iran,BIQ037/2,,27\n قاقز\n / \n929\n ةلحم\n /\n تاصرعلا\n /\n...,Baghdad / Al Arasat / sectiom 929 / lane 27
4,نايسراب,Barsian,BIQ052/2,يملاسلاا ينطولا فرصملا لباقم\n /,69\n ةيانب\n /\n30\n عراش\n / \n929\n ةلحم\n /...,Baghdad / Karrada / Babil neighborhood / Al Ar...
5,تارفلاو هلجد,Dijlah wa Al Furat,EIQ0255/3,,19\n قاقز\n / \n110\n ةلحم\n /\n ديشرلا عراش\n...,Baghdad / Al Rasheed street / section 110 / la...
...,...,...,...,...,...,...
104,,Afaq Dubai,,,,Nishtiman Exchange Market
105,,Reydin,,,,Sheikhallah Exchange Market
106,,Al Khalidi Exchange,,,,Irbil / New Borsa / Nishtaman Building second ...
107,ءاطعلا,Al Attaa,BIQ034/2,,ةيرحلا ةحاس برق\n /\n ةداركلا\n /\n دادغب,Baghdad / Karrada / near Al Horriya square


In [48]:
import hashlib
from datetime import datetime
last_updated_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
def get_hash(n):
    return hashlib.sha256(((n+"Central Bank of Iraq Public Black List, Iraq" + "IRQ_T30064").lower()).encode()).hexdigest()


In [53]:
out_list1 = []
for i,row in main.astype(str).iterrows():
    pass_dict = {}
    name = row["English Name\n يزيلجنلااب مسلاا"]
    code = row["Code\nزمرلا"]
    pass_dict["uid"] = get_hash(code+name)
    pass_dict["name"] = name

    pass_dict["alias_name"] = [row["Arabic Name\n يبرعلاب مسلاا"]]

    pass_dict["list_type"] = "Entity"
    pass_dict["nns_status"] = "False"
    pass_dict["last_updated"] = last_updated_string
    pass_dict["list_id"] = "IRQ_T30064"
    pass_dict["country"] = ["Iraq"]

    comp = row["English Address\nيزيلجنلااب ناونعلا"]
    comp = comp.split(" / ")
    comp.reverse()
    comp = ",".join(comp)
    pass_dict["address"] = []
    pass_dict["address"].append({
        "complete_address": comp,
        "country" : "Iraq"
    })
    pass_dict["sanction_details"] = {}
    pass_dict["entity_details"] = {}
    pass_dict["comment"] = ""
    pass_dict["sanction_list"] = {}
    pass_dict["sanction_list"] = {
            "sl_authority": "Central Bank of Iraq, Iraq",
            "sl_url": "https://cbi.iq/news/view/741",
            "watch_list": "EMEA Watchlists",
            "sl_host_country": "Iraq",
            "sl_type": "Sanctions",
            "sl_source": "Central Bank of Iraq Public Black List, Iraq",
            "sl_description": "List of individuals and entities sanctioned by Central Bank of Iraq, Iraq"
        }

    out_list1.append(pass_dict)
    

In [50]:
out_list1[-1]
len(out_list1)


108

In [15]:
import pandas as pd
main = pd.read_excel("raw-files/Book 1.xlsx")

In [25]:
in_main = main[0:94]
en_main = main[95:]
en_main


,أسماء شركات التوسط,Name,Code,Arabic,English,Day,Month,Year
95,سلسلة الذهب,Selselat Al Thahab,TIQ026/1,كربﻻء,Karbala,15.0,5.0,2016.0
96,النبال العربية,Al Nibal Al Arabia,TIQ001/1,بغداد,Baghdad,8.0,1.0,2016.0
97,النهر الخالد,Al Nahr Al Khalid,TIQ004/1,بغداد,Baghdad,6.0,6.0,2017.0
98,الوائل,Al Wa'eel,TIQ019/1,بغداد,Baghdad,15.0,5.0,2016.0
99,مصرف العطاء اﻻسﻻمي (البﻻد سابقا),Al Ataa Islamic Bank,BIQ034/2,بغداد,Baghdad,16.0,5.0,2018.0
100,مصرف التعاون اﻷسﻻمي لﻷستثمار,Islamic Cooperation Bank for Investment,BIQ036/2,بغداد,Baghdad,8.0,1.0,2016.0
101,بنك ملي ايران,Bank Melli Iran,BIQ037/2,بغداد,Baghdad,8.0,1.0,2016.0
102,مصرف نور العراق اﻻسﻻمي لﻻستثمار \nوالتمويل,Iraq Noor Islamic Bank \nfor Investment & Fin...,BIQ064/2,بغداد,Baghdad,30.0,10.0,2019.0
103,بنك بارسيان,Parsian Bank,BIQ052/2,بغداد,Baghdad,8.0,1.0,2016.0
104,مصرف بابل,Babylon Bank,BIQ019/2,بغداد,Baghdad,24.0,12.0,2020.0


In [28]:
main.columns

Index(['أسماء شركات التوسط ', 'Name', 'Code', 'Arabic ', 'English', 'Day',
       'Month', 'Year'],
      dtype='object')

In [44]:
def tab_scrap(m,t):
    pass_list = []
    for i,r in m.astype(str).iterrows():
        name = r["Name"]
        code = r["Code"]
        uid = get_hash(name+code)

        alias = [r["أسماء شركات التوسط "]]

        comp = r["English"]
        adrs = [{
            "complete_address" : f"{comp.strip()},Iraq",
            "country" : "Iraq"
        }]

        date  = f"{r['Day']}/{r['Month']}/{r['Year']}"
        
        if t == "e":
            pass_list.append(
            {
                "uid":uid,
                "name":name,
                "alias_name": alias,
                "country": ["Iraq"],
                "list_type": "Entity",
                "last_updated": last_updated_string,
                "list_id": "IRQ_T30064",
                "entity_details": {},
                "nns_status": "False",
                "address": adrs,
                "sanction_details": {
                    "date" : date 
                                    },
                "comment":"",
                "sanction_list": {
                    "sl_authority": "Central Bank of Iraq, Iraq",
                    "sl_url": "https://cbi.iq/news/view/741",
                    "watch_list": "EMEA Watchlists",
                    "sl_host_country": "Iraq",
                    "sl_type": "Sanctions",
                    "sl_source": "Central Bank of Iraq Public Black List, Iraq",
                    "sl_description": "List of individuals and entities sanctioned by Central Bank of Iraq, Iraq"
                }
            }
            )
        else:   
            pass_list.append(
            {
                "uid":uid,
                "name":name,
                "alias_name": alias,
                "country": ["Iraq"],
                "list_type": "Individual",
                "last_updated": last_updated_string,
                "list_id": "IRQ_T30064",
                "individual_details": {},
                "nns_status": "False",
                "address": adrs,
                "sanction_details": {
                    "date" : date 
                                    },
                "comment":"",
                "sanction_list": {
                    "sl_authority": "Central Bank of Iraq, Iraq",
                    "sl_url": "https://cbi.iq/news/view/741",
                    "watch_list": "EMEA Watchlists",
                    "sl_host_country": "Iraq",
                    "sl_type": "Sanctions",
                    "sl_source": "Central Bank of Iraq Public Black List, Iraq",
                    "sl_description": "List of individuals and entities sanctioned by Central Bank of Iraq, Iraq"
                }
            }
            )
    return pass_list
        

In [45]:
out1 = tab_scrap(en_main,"e")
out2 = tab_scrap(in_main,"i")
print(len(out1))
print(len(out2))

10
94


In [54]:
main_out = out_list1 + out1 + out2
len(main_out)

212

In [58]:
import json
with open("sng-711.json", "w",encoding='utf-8') as outfile:
    json.dump(main_out, outfile, ensure_ascii=False, indent=4)